In [1]:
import os, findspark

#  Spark 4.0.1
os.environ["SPARK_HOME"] = "/opt/spark-4.0.1-bin-hadoop3"
os.environ["PATH"] = os.path.join(os.environ["SPARK_HOME"], "bin") + ":" + os.environ["PATH"]


findspark.init("/opt/spark-4.0.1-bin-hadoop3")

# create Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("JobPostingsAnalysis").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/02 01:17:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Load CSV file into Spark DataFrame

df = (
    spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .option("multiLine", "true")
        .option("escape", "\"")
        .csv("data/lightcast_job_postings.csv")
)

In [3]:
df.count()

72498

In [4]:
df.show(5)
len(df.columns)

25/12/02 01:18:18 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----------------+----------------------+----------+--------+---------+--------+--------------------+--------------------+--------------------+-----------+-------------------+--------------------+--------------------+---------------+----------------+--------+--------------------+-----------+-------------------+----------------+---------------------+-------------+-------------------+-------------+------------------+---------------+--------------------+--------------------+--------------------+-------------+------+-----------+----------------+-------------------+---------+-----------+--------------------+--------------------+-------------+------+--------------+-----+--------------------+-----+----------+---------------+--------------------+---------------+--------------------+------------+--------------------+------------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+------

131

In [5]:
df.printSchema() 

root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: string (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: timestamp (nullable = true)
 |-- DUPLICATES: integer (nullable = true)
 |-- POSTED: string (nullable = true)
 |-- EXPIRED: string (nullable = true)
 |-- DURATION: integer (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: string (nullable = true)
 |-- MODELED_DURATION: integer (nullable = true)
 |-- COMPANY: integer (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: boolean (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nullable = true)
 |-- MIN_EDULEVELS: integer (

# **Data Cleaning & Preprocessing**

### **Which columns are irrelevant or redundant?**

The original dataset contains 131 variables, including job information, industry codes, geographic identifiers, tracking fields, and Lightcast-specific coded values. 
These fields either contain redundant information or lack interpretability; therefore, they are removed from the cleaned dataset.

1. **Multiple levels and historical versions of NAICS and SOC codes**

e.g., NAICS_2022_2, NAICS_2022_3, SOC_2021_2, which represent the same classification system with redundant layers.

2. **Location fields that contain encoded strings or latitude–longitude formats**

e.g., CITY, LOCATION, MSA_NAME_OUTGOING, rather than readable city or county names.

3. **Lightcast proprietary coded fields**

e.g., ONET, CIP, LOT_V6_ series, which only include numerical codes whose meaning cannot be interpreted directly. Since readable name fields exist, these numerical versions are unnecessary.

4. **Tracking and metadata fields**
 
such as ACTIVE_URLS, LAST_UPDATED_TIMESTAMP, and SOURCE_TYPE, which are internal system variables that do not contribute to labor market analysis.

5. **Non-analytical text fields**

e.g., MODELED_EXPIRED, DURATION unrelated to our wage, industry, or geographic comparisons.


***


## **Why are we removing multiple versions of NAICS/SOC codes?**

The dataset includes numerous versions and hierarchical levels of industry (NAICS) and occupation (SOC) codes. These repeated layers introduce several issues:

- They duplicate the same industry/occupation information across different levels.

- They may cause the same job posting to be categorized multiple times.

- They introduce unnecessary dimensions for modeling and visualization.

- They make results more difficult to interpret due to inconsistent classification levels.

To ensure clarity and standardization, we keep only:

1. NAICS_2022_6_NAME (latest and most granular industry classification) 
2. SOC_2021_4_NAME (consistent and interpretable occupation classification)

In [6]:
columns_to_drop = [
    "ID", "URL", "ACTIVE_URLS", "DUPLICATES", "LAST_UPDATED_TIMESTAMP",
    "NAICS2", "NAICS3", "NAICS4", "NAICS5", "NAICS6",
    "SOC_2", "SOC_3", "SOC_4", "SOC_5",
    "NAICS_2022_2", "NAICS_2022_3","NAICS_2022_4","NAICS_2022_5",
    "NAICS_2022_2_NAME","NAICS_2022_3_NAME","NAICS_2022_4_NAME","NAICS_2022_5_NAME",
    "SOC_5_NAME","SOC_4_NAME","SOC_3_NAME","SOC_2_NAME",
    "NAICS2_NAME","NAICS3_NAME","NAICS4_NAME","NAICS5_NAME",
    "LAST_UPDATED_DATE","EXPIRED","DURATION",    # date variables are not related to our topic 
    "MODELED_EXPIRED","MODELED_DURATION","MODELED_EXPIRED",
    "COMPANY", "TITLE", "SKILLS",                  # remove numerical COMPANY
    "MAX_EDULEVELS","MAX_EDULEVELS_NAME",
    "EMPLOYMENT_TYPE","REMOTE_TYPE", "EMPLOYMENT_TYPE",     # remove numerical EMPLOYMENT_TYPE, REMOTE_TYPE, keeping the text version
    "LOCATION", "CITY","COUNTY","MSA","STATE", 
    "MSA_NAME", "COUNTY_NAME_OUTGOING", "COUNTY_NAME_INCOMING", "MSA_NAME_OUTGOING", "MSA_NAME_INCOMING",   
    "COUNTY_OUTGOING","COUNTY_INCOMING", "MSA_OUTGOING", "MSA_INCOMING",         # LOCATION contains latitude and longitude coordinates, CITY ontains encoded strings instead of actual city names 
    "EDUCATION_LEVELS", "MIN_EDULEVELS",  # encoded strings instead of actual EDUCATION_LEVELS, we keep EDUCATION_LEVELS_NAME. MIN_EDULEVELS contains number 
    "SPECIALIZED_SKILLS", "CERTIFICATIONS", "COMMON_SKILLS", "SOFTWARE_SKILLS",    # encoded strings
    "SOC_2021_2", "SOC_2021_2_NAME", "SOC_2021_3", "SOC_2021_3_NAME", "SOC_2021_5", "SOC_2021_5_NAME",
    "LOT_CAREER_AREA", "LOT_OCCUPATION", "LOT_SPECIALIZED_OCCUPATION", "LOT_OCCUPATION_GROUP","LOT_V6_SPECIALIZED_OCCUPATION", "LOT_V6_CAREER_AREA", "LOT_V6_OCCUPATION_GROUP",  
    "ACTIVE_SOURCES_INFO", "MAX_YEARS_EXPERIENCE", "LIGHTCAST_SECTORS", "LIGHTCAST_SECTORS_NAME", "ORIGINAL_PAY_PERIOD",   # missing values >50%
    "SOURCE_TYPES", "SOURCES", "BODY", 
    "COMPANY_NAME", "TITLE_NAME",  #keep  COMPANY_RAW and TITLE_CLEAN (simplified version)
    "ONET", "ONET_2019","CIP6", "CIP4", "CIP2",       # numerical 
    "CIP6_NAME","CIP4_NAME","CIP2_NAME",     # remove zipcode-related variables, we keep location name from state to county
    
] 

df = df.drop(*columns_to_drop)

In [7]:
df.show(5)

len(df.columns)

+--------+--------------------+-----------+-------------------+---------------------+-------------------+--------------------+--------------------+-------------+------+----------------+---------+-----------+-------------+--------------+----------+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+---------------+--------------------+--------------------+-------------------------------+-------------------------+----------------------------------+-----------------+----------------------+----------------------------+-----------------------+------------+--------------------+
|  POSTED|           TITLE_RAW|COMPANY_RAW|COMPANY_IS_STAFFING|EDUCATION_LEVELS_NAME| MIN_EDULEVELS_NAME|EMPLOYMENT_TYPE_NAME|MIN_YEARS_EXPERIENCE|IS_INTERNSHIP|SALARY|REMOTE_TYPE_NAME|SALARY_TO|SALARY_FROM|    CITY_NAME|   COUNTY_NAME|STATE_NAME|         NAICS6_NAME|  

38

## **3. How will this improve analysis?**

After cleaning, the dataset is reduced from 131 variables to 38 variables, which brings several advantages:

1. **Improved interpretability**

- Removing encoded fields (e.g., CITY with codes, LOCATION with coordinates, Lightcast numeric IDs) ensures that all remaining variables contain readable, meaningful text such as state names, county names, industry names, and occupation names.

- These fields can be directly used for mapping, comparison, and reporting.

2. **Less noise, more focused analysis**

- Eliminating irrelevant or repetitive fields ensures that our wage analysis, industry structure comparisons, and geographic insights remain clean and consistent.

- Keeping only the most relevant variables avoids duplicate counting across different NAICS/SOC layers.

3. **Higher modeling efficiency**

- A smaller and cleaner feature set reduces the need for extensive dummy encoding.

- It minimizes risks of multicollinearity and improves model stability.

- Feature engineering becomes faster and more controlled.

4. **Cleaner structure for visualization and reporting**

- With only meaningful fields preserved, charts and tables become easier to interpret.

- All retained variables directly support our analytical goals.


In [8]:
from pyspark.sql import functions as F

total_rows = df.count()

exprs = [
    F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df.columns
]

missing_row = df.agg(*exprs).collect()[0].asDict()

missing_stats = [(col, missing_row[col], missing_row[col] / total_rows)
                 for col in df.columns]

missing_df = spark.createDataFrame(
    missing_stats,
    ["column", "missing_count", "missing_ratio"]
)

missing_df.orderBy("missing_ratio", ascending=False).show(60)

+--------------------+-------------+--------------------+
|              column|missing_count|       missing_ratio|
+--------------------+-------------+--------------------+
|              SALARY|        41690|  0.5750503462164474|
|           SALARY_TO|        40100|   0.553118706722944|
|         SALARY_FROM|        40100|   0.553118706722944|
|MIN_YEARS_EXPERIENCE|        23146| 0.31926397969599163|
|         COMPANY_RAW|          541|0.007462274821374383|
|         TITLE_CLEAN|          140|0.001931087754144942|
|           TITLE_RAW|          104|0.001434522331650...|
|SPECIALIZED_SKILL...|           44|6.069132941598389E-4|
| COMPANY_IS_STAFFING|           44|6.069132941598389E-4|
| CERTIFICATIONS_NAME|           44|6.069132941598389E-4|
|EDUCATION_LEVELS_...|           44|6.069132941598389E-4|
|  COMMON_SKILLS_NAME|           44|6.069132941598389E-4|
|  MIN_EDULEVELS_NAME|           44|6.069132941598389E-4|
|SOFTWARE_SKILLS_NAME|           44|6.069132941598389E-4|
|EMPLOYMENT_TY

# Handling Missing Values

- Numerical fields (e.g., Salary) are filled with the median.

- Categorical fields (e.g., Industry) are replaced with "Unknown".

- Columns with >50% missing values are dropped. (We already finished that by dropping un-relevant conlumns', but salary varaibles are super important so we still keep it and we will see whatever we can do later.)

In [9]:
# Fill numerical missing values
median_experience = df.approxQuantile("MIN_YEARS_EXPERIENCE", [0.5], 0.01)[0]
df = df.fillna({"MIN_YEARS_EXPERIENCE": median_experience})

In [10]:
# Fill categorical missing values
df = df.fillna({"NAICS_2022_6_NAME": "Unknown"})

In [11]:
from pyspark.sql import functions as F

total_rows = df.count()

# Compute missing count for each column
exprs = [
    F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df.columns
]

missing_row = df.agg(*exprs).collect()[0].asDict()

# Convert into list of (column, missing_count)
missing_stats = [(col, missing_row[col]) for col in df.columns]


missing_df = spark.createDataFrame(
    missing_stats,
    ["column", "missing_count"]
)


missing_df.orderBy("missing_count", ascending=False).show(60)

+--------------------+-------------+
|              column|missing_count|
+--------------------+-------------+
|              SALARY|        41690|
|           SALARY_TO|        40100|
|         SALARY_FROM|        40100|
|         COMPANY_RAW|          541|
|         TITLE_CLEAN|          140|
|           TITLE_RAW|          104|
|SPECIALIZED_SKILL...|           44|
| COMPANY_IS_STAFFING|           44|
| CERTIFICATIONS_NAME|           44|
|EDUCATION_LEVELS_...|           44|
|  COMMON_SKILLS_NAME|           44|
|  MIN_EDULEVELS_NAME|           44|
|SOFTWARE_SKILLS_NAME|           44|
|EMPLOYMENT_TYPE_NAME|           44|
|           ONET_NAME|           44|
|       IS_INTERNSHIP|           44|
|      ONET_2019_NAME|           44|
|    REMOTE_TYPE_NAME|           44|
|          SOC_2021_4|           44|
|           CITY_NAME|           44|
|     SOC_2021_4_NAME|           44|
|         COUNTY_NAME|           44|
|LOT_CAREER_AREA_NAME|           44|
|          STATE_NAME|           44|
|